In [1]:
from datetime import *
import pandas as pd
import glob
import warnings
import os
from os.path import *
from datetime import datetime

In [2]:
warnings.filterwarnings('ignore')

In [3]:
data_path = '/Users/kakusou/Desktop/Crypto/data/'

# for file in os.listdir(data_path):
#     if file=='.DS_Store': continue
#     df = pd.read_csv(join(data_path, file))
#     # translate time
#     df['time'] = pd.to_datetime(df['Open time']/1000, unit='s')
#     df = df[['time','Open', 'High', 'Low', 'Close', 'Volume']]
#     df = df.sort_values(by='time')
#     df = df.reset_index(drop=True)
#     # save
#     df.to_csv(join(data_path, "1_"+file),index=False)

# csv -> Format 1

In [4]:
# train
train_start_time = "2019-01-31 16:00:00"
train_end_time = "2021-04-19 00:00:00"
# backtest
backtest_start_time = "2021-04-19 00:00:00"
backtest_end_time = "2021-05-03 00:00:00"

In [5]:
train_format1_path = join(data_path, 'train_data_format1_week_final.h5')
train_format2_path = join(data_path, 'train_data_format2_week_final.h5')
backtest_format1_path = join(data_path, 'backtest_data_format1_week_final.h5')
backtest_format2_path = join(data_path, 'backtest_data_format2_week_final.h5')

In [6]:
train_h5data = pd.HDFStore(train_format1_path)
backtest_h5data = pd.HDFStore(backtest_format1_path)

In [7]:
from os import listdir
from os.path import isfile, join
file_list = [f for f in listdir(data_path) if isfile(join(data_path, f)) and '.csv' in f] 
file_list

['LTCUSDT_1m_20190201_20210503.csv',
 'XRPUSDT_1m_20190201_20210503.csv',
 'BTCUSDT_1m_20190201_20210503.csv',
 'ETHUSDT_1m_20190201_20210503.csv']

In [8]:
for file in file_list:
    symbol = file.split(sep="_")[0]+'.csv'
    df = pd.read_csv(data_path + file, parse_dates=["time"])
    train_df = df[(df.time >= train_start_time) & (df.time < train_end_time)]
    backtest_df = df[(df.time >= backtest_start_time) & (df.time < backtest_end_time)]
    train_df.set_index(keys="time", drop=True, inplace=True)
    backtest_df.set_index(keys="time", drop=True, inplace=True)
    train_h5data[symbol] = train_df
    backtest_h5data[symbol] = backtest_df
    print(symbol)

LTCUSDT.csv
XRPUSDT.csv
BTCUSDT.csv
ETHUSDT.csv


In [9]:
train_h5data.close()
backtest_h5data.close()

# Format 1 -> Format 2

In [10]:
import h5py

In [11]:
def read_h5(path, dtype='dataframe'):
    assert dtype in ['dataframe', 'nparray'], "You didn't get the right dtype"
    dataList = list()
    keys = list()
    
    f = h5py.File(path, 'r')
    for key in f.keys():
        keys.append(key)
    f.close()
    
    for key in keys:
        if dtype=='dataframe':
            dataList.append(pd.read_hdf(path, key))
        else:
            dataList.append(pd.read_hdf(path, key).values)
                        
    return dataList, keys

In [12]:
from tqdm import tqdm
def convert_format(format1_path, format2_path):
    (l, keys) = read_h5(format1_path)
    index = l[0].index.unique()
    h5f = h5py.File(format2_path, 'w')
    for i in tqdm(index):
        min_data = l[0].loc[[i]]
        for j in range(1, len(l)):
            min_data = min_data.append(l[j].loc[[i]])
        h5f.create_dataset(str(i), data=min_data.values)
    h5f.close()

In [13]:
convert_format(backtest_format1_path, backtest_format2_path)

100%|██████████| 19566/19566 [01:00<00:00, 322.20it/s]
